In [1]:
from pathlib import Path
import pandas as pd

In [2]:
data = pd.read_csv("meta_dataset.csv")
X_train = data[["n_samples", "n_features", "feature_ratio", "target_std", "target_skew", "mean_feature_skew", "mean_feature_kurtosis", "zero_var_pct", "mean_abs_corr", "max_abs_corr"]].copy()
X_train

,n_samples,n_features,feature_ratio,target_std,target_skew,mean_feature_skew,mean_feature_kurtosis,zero_var_pct,mean_abs_corr,max_abs_corr
0,7038,11,0.001563,181.297663,1.261745,-0.018617,-1.164749,0.0,0.195733,0.990518
1,4329,11,0.002541,0.800043,0.309079,1.897983,2.048831,0.0,0.460580,0.987421
2,8611,81,0.009407,34.085346,0.843986,3.596372,13.575943,0.0,0.344543,0.997759
3,3946,12,0.003041,0.878720,0.203930,2.541719,4.949163,0.0,0.250987,0.726846
4,7196,62,0.008616,0.030384,-2.791005,2.408849,4.708734,0.0,0.169848,0.989871


In [26]:
y_train = data[["LGBMRegressor_rank", "XGBRegressor_rank", "RandomForestRegressor_rank", "LinearRegression_rank", "SVR_rank"]].copy()
y_train
# y_train_1s = (y_train == 1).astype(int)
# y_train_1s

,LGBMRegressor_rank,XGBRegressor_rank,RandomForestRegressor_rank,LinearRegression_rank,SVR_rank
0,1,2,3,4,5
1,3,1,2,5,4
2,2,3,1,5,4
3,3,2,1,5,4
4,1,4,2,3,5


In [4]:
loss = data[["LGBMRegressor_r2", "XGBRegressor_r2", "RandomForestRegressor_r2", "LinearRegression_r2", "SVR_r2"]].copy()
1 - loss 

,LGBMRegressor_r2,XGBRegressor_r2,RandomForestRegressor_r2,LinearRegression_r2,SVR_r2
0,0.080955,0.090957,0.104048,0.612358,0.699272
1,0.038476,0.006241,0.014897,0.134079,0.089747
2,0.105055,0.106375,0.101163,0.279409,0.261764
3,0.534863,0.516746,0.473426,0.685085,0.631429
4,0.924151,0.990069,0.930828,0.962272,1.295200


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent.parent))  # Add project root to sys.path

from src.automl.meta_nn import MetaNet

# Assume X_train and y_train are pandas DataFrames
X_np = X_train.values.astype(np.float32)
y_np = y_train_1s.values.astype(np.float32)

X_tensor = torch.from_numpy(X_np)
y_tensor = torch.from_numpy(y_np)

model = MetaNet(X_tensor.shape[1], y_tensor.shape[1])

# Training setup
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary targets
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 2

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/2, Loss: 169.8965


In [25]:
with torch.no_grad():
    logits = model.forward(X_tensor[0:1])
    probs = torch.softmax(logits, dim=1)
    print(probs)


tensor([[1., 0., 0., 0., 0.]])
